In [1]:
import pandas as pd
import jsonlines
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import FastText
import fasttext
import fasttext.util
import fasttext.util
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Загрузим данные
data = []

with jsonlines.open('C:/data.jsonl', 'r') as reader:
    for line in reader:
        data.append(line)
        
# Создаем DataFrame из данных
df = pd.DataFrame(data)

# Предварительный анализ данных
print(df.head())

                                                text  label
0  i feel awful about it too because it s my job ...      0
1                              im alone i feel awful      0
2  ive probably mentioned this before but i reall...      1
3           i was feeling a little low few days back      0
4  i beleive that i am much more sensitive to oth...      2


In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Очистка текста от лишних символов и токенизация

df['text'] = df['text'].str.replace(r'[^a-zA-Z]', ' ')
df['text'] = df['text'].str.lower()
df['text'] = df['text'].apply(word_tokenize)
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join(word for word in text if word not in stop_words)

df['text'] = df['text'].apply(remove_stopwords)

C:\Users\user\AppData\Local\Temp/ipykernel_12684/1455451783.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r'[^a-zA-Z]', ' ')


In [5]:
# Векторизация текста с использованием TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(df['text'])
y = df['label']

In [6]:
# Разделение данных на обучающий и тестовый наборы и обучение модели
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [7]:
# Оценка модели и предоставление метрик
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred)

print(f'F1 Score: {f1}')
print(report)

F1 Score: 0.8988322173496703
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     24504
           1       0.92      0.93      0.93     28247
           2       0.80      0.77      0.79      6853
           3       0.89      0.90      0.90     11339
           4       0.85      0.85      0.85      9376
           5       0.79      0.71      0.75      3043

    accuracy                           0.90     83362
   macro avg       0.86      0.85      0.86     83362
weighted avg       0.90      0.90      0.90     83362



In [8]:
fasttext.util.download_model('en', if_exists='ignore')  # English
fasttext_model = fasttext.load_model('cc.en.300.bin')

In [11]:
# Загрузим данные
data = []

with jsonlines.open('C:/data.jsonl', 'r') as reader:
    for line in reader:
        data.append(line)
        
# Создаем DataFrame из данных
df = pd.DataFrame(data)

# Предварительный анализ данных
print(df.head())

                                                text  label
0  i feel awful about it too because it s my job ...      0
1                              im alone i feel awful      0
2  ive probably mentioned this before but i reall...      1
3           i was feeling a little low few days back      0
4  i beleive that i am much more sensitive to oth...      2


In [12]:
# Функция для создания эмбеддингов для текстов
def text_to_vector(text):
    words = text.split()
    vectors = [fasttext_model[word] for word in words if word in fasttext_model]
    if not vectors:
        return [0] * fasttext_model.vector_size
    return sum(vectors) / len(vectors)

# Преобразование текстов в эмбеддинги
df['text_vector'] = df['text'].apply(text_to_vector)

In [13]:
# Класс для простой нейронной сети + LSTM слой
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x, _ = self.lstm(x)
        x = self.fc2(x)
        return x


# Преобразование данных в тензоры PyTorch
X = torch.Tensor(df['text_vector'].tolist())
y = torch.LongTensor(df['label'].values)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание даталоадеров
train_data = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)

C:\Users\user\AppData\Local\Temp/ipykernel_12684/1317166703.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  X = torch.Tensor(df['text_vector'].tolist())


In [14]:
# Создание экземпляра нейронной сети
input_size = X_train.shape[1]
hidden_size = 128
output_size = len(df['label'].unique())

model = SimpleNN(input_size, hidden_size, output_size)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
num_epochs = 20
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Оценка модели на тестовых данных
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    
# Преобразование тензоров PyTorch обратно в массивы NumPy
y_test_numpy = y_test.numpy()
predicted_numpy = predicted.numpy()

# Вычисление метрик и создание отчета о классификации
f1 = f1_score(y_test_numpy, predicted_numpy, average='weighted')
report = classification_report(y_test_numpy, predicted_numpy)

print(f'F1 Score: {f1}')
print(report)

F1 Score: 0.7047659248975529
              precision    recall  f1-score   support

           0       0.73      0.77      0.75     24504
           1       0.72      0.82      0.77     28247
           2       0.64      0.49      0.55      6853
           3       0.73      0.62      0.67     11339
           4       0.65      0.58      0.61      9376
           5       0.59      0.42      0.49      3043

    accuracy                           0.71     83362
   macro avg       0.68      0.62      0.64     83362
weighted avg       0.71      0.71      0.70     83362



## Вывод

### Первая модель:
* F1: 0.8988
* Точность и полнота высоки для всех классов (в пределах 0.80-0.94).
* Макро-средний F1: 0.86
* Взвешенный средний F1: 0.90
В целом, модель показывает отличные результаты с высокой точностью и полнотой для всех классов. Взвешенный средний F1близок к 1, что указывает на хорошую способность модели обобщать и классифицировать данные.

### Вторая модель:
* F1: 0.7048
* Точность и полнота ниже для всех классов (в пределах 0.59-0.73).
* Макро-средний F1: 0.64
* Взвешенный средний F1: 0.70
Модель во втором случае показывает намного более низкие метрики по сравнению с первым случаем. Она имеет более низкую точность и полноту, что может означать, что она делает больше ошибок в классификации данных.

### Анализ:
Плохие результаты второй модели можно объяснить разными архитектурами моделей, а так же отсутсвие предобработки данных на второй модели. Повысить результаты второй модели можно, включив предобработку данных, а также изменяя гиперпараметры.